## Пример 1: Антивирусное программное обеспечение

Пример адаптивной антивирусной системы на **Python** предполагает моделирование процесса использования _машинного обучения_ для обнаружения потенциально ___вредоносных файлов___ на основе моделей поведения, а не только на основе известных *вирусных сигнатур*. Этот пример будет представлять собой базовую демонстрацию с использованием _синтетических данных_.

### Шаг 1: Импорт библиотек и создание необходимых инструментов

In [ ]:
import numpy as np # Библиотека для работы с данными
import pandas as pd # Библиотека для работы с данными
from sklearn.linear_model import LogisticRegression # Модель обучения: логистическая регрессия
from sklearn.model_selection import train_test_split # разделитель данных для обучения модели
from sklearn.metrics import accuracy_score # подсчет эффективности модели
import os # библиотека для работы с операционной системой
import plotly.express as px # Инструмент для работы с интерактивными графиками
from binary_list import BinaryList # библиотека для работы со списком бинарных сигнатур файлов
from utilities import read_file # Функция read_file считает содержимое файла по указанному ей пути
from utilities import count_bits_in_file # Функция подсчета бит = 1 и бит = 1 идущих подряд с конца файла
from utilities import data_for_analysis # Фнкция преобразования данных в формат для анализа
from utilities import MakeScatter # инструмент создания интерактивного графика

Для начала посмотрим на список файлов и директорий с которыми нам предстоит работать

In [ ]:
import subprocess
# Выполнение команды tree и захват вывода
result = subprocess.run(['tree', '/f', '/a', 'AdaptiveAntivirus'], stdout=subprocess.PIPE, text=True, shell=True)
# Печать результатов
print(result.stdout)

#### Добавление функций и путей к файлам

In [ ]:
normal_files_dir = os.path.join("AdaptiveAntivirus", "normal_files") # путь к обычным файлам сохраним в переменную normal_files_dir
virus_files_dir = os.path.join("AdaptiveAntivirus", "viruses") # путь к файлам в которых точно известно что есть вирус сохраним
# в переменную virus_files_dir
suspect_files_dir = os.path.join("AdaptiveAntivirus", "suspects") # путь к файлам в которых подозревается наличие вируса

In [ ]:
normal_files_dir # посмотрим путь к директории с обычными файлами

In [ ]:
virus_files_dir # посмотрим путь к директории с точно вирусными файлами

In [ ]:
os.listdir(normal_files_dir) # покажет список всех файлов в директории с обычными файлами normal_files_dir

### Шаг 2: Создаем списки сигнатур нормальных файлов, вирусных файлов и подозрительных файлов
#### Создаём список сигнатур нормальных файлов

In [ ]:
normal_files = [os.path.join(normal_files_dir, f) for f in os.listdir(normal_files_dir)] # сохраняем список относительных путей
# файлов os.listdir(normal_files_dir) в переменную normal_files
normal_data = BinaryList([read_file(f) for f in normal_files]) # сохраняем в переменную normal_data содержимое каждого файла
# директории normal_files_dir

In [ ]:
normal_data[:5] # посмотрим бинарные сигнатуры первых 5 файлов директории normal_files_dir

#### Создаём список сигнатур вирусных файлов

In [ ]:
virus_files = [os.path.join(virus_files_dir, f) for f in os.listdir(virus_files_dir)]
virus_data = BinaryList([read_file(f) for f in virus_files]) # сохраняем в переменную virus_data сигнатуры вирусных файлов

In [ ]:
virus_data # посмотрим на получившееся сигнатуры

**Обратите внимание** на то как отличаются сигнатуры вирусных файлов и обычных файлов

#### Создаём список сигнатур подозрительных файлов

In [ ]:
suspect_files = [os.path.join(suspect_files_dir, f) for f in os.listdir(suspect_files_dir)]
suspect_data = BinaryList([read_file(f) for f in suspect_files]) # сохраняем в переменную virus_data сигнатуры вирусных файлов

In [ ]:
suspect_data # посмотрим на получившееся сигнатуры

### Шаг 3: Анализируем данные графически.

#### Комбинируем сигнатуры нормальных, вирусных файлов и подозрительных

In [ ]:
data = normal_data + virus_data + suspect_data
labels =[0] * len(normal_data) + [1] * len(virus_data) + [2]*len(suspect_data) # создаем разметку, где 0 это не вирус, а 1 это вирус, 2 - подозрительный

In [ ]:
BinaryList(data[:5]), BinaryList(data[-5:]) # Прверяем сформированный список: первые 5 и последниые 5 сигнатур

In [ ]:
print(*labels) # псомотрим на разметку. Первыми идут нормальные файлы далее шесть считаем вирусами и 3 последних подозрительными

In [ ]:
data_for_analysis(data) # посмотрим на преобразование сигнатуры в числа.

**Посмотрите на получившиеся числа** 
Возьмем для примера первый файл (afdsl.bak) bits: 1110010000001001 и первый файл вируса bits: 0000001111111111

``[6, 1, 0]`` 6 - это то, сколько всего бит = 1; 1 - сколько подряд идущих бит = 1 с конца сигнатуры bits: **111**00**1**000000**1**00**1**

``[10, 10, 6]``  6 - это то, сколько всего бит = 0 с начала сигнатуры bits: **000000**1111111111

In [ ]:
features = np.array(data_for_analysis(data)) # сохраним в переменную features преобразованные сигнатуры

In [ ]:
scatter_plot = MakeScatter(features, labels)
scatter_plot.show()

Обратите внимание на то, как расположились подозрительные файлы, нормальные файлы и вирусные файлы. **Тут явно есть некоторая закономерность**

Теперь нам нужно обучить модель и посмотреть поймет ли она что такое подозрительные файлы. Обучать будем только на нормальных и вирусных файлах

### Шаг 4: Тренируем модель

Используем только нормальные файлы и вирусные

In [ ]:
data = normal_data + virus_data
labels =[0] * len(normal_data) + [1] * len(virus_data) # создаем разметку, где 0 это не вирус, а 1 это вирус

#### Создадим модель и обучим её

In [ ]:
X = np.array(data_for_analysis(data)) # анализируемые параметры
y = np.array(labels) # метки параметров: вирус, не вирус
# Разделение данных на обучающую и тестовую выборку
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Создание и обучение модели
model = LogisticRegression()
model.fit(X_train, y_train)

# Предсказание на тестовых данных
y_pred = model.predict(X_test)

# Оценка точности
accuracy = accuracy_score(y_test, y_pred)
print('Точность модели:', accuracy)

### Шаг 5: Исследуем файлы приходящие в нашу сеть

В директории AdaptiveAntivirus, есть каталог **internet_files** в нем есть несколько файлов:

1. bggtzpabxt.txt
2. pzmdwtxxdu.bak
3. sdxzpoqtaw.txt
4. ooqoiadwyp.dat
5. ptysmowwuh.log
6. dimwhytalt.txt
7. vogwzlinth.bak
8. jxwyevqeps.bin
9. qmlmsfyeny.txt
10. omhfqwlqgf.txt

Нужно проверить каждый через обученную модель. И ответить на вопрос: ***Является файл вирусом?***

#### Далее идет псевдокод, который нужно преобразовать в программу на языке Python

Основные функции которые нужно использовать. Ниже они же в Псевдокоде:

1. read_file( <имя_файла> ) - прочитать файл.
```
считать_файл( )
```
2. os.path.join( '<имя_каталога1>','<имя_каталога2>', '<имя_файла>' ) - создать путь по правилам операционной системы. 
```
путь_к_файлу( )
```
3. data_for_analysis(\[ <Данные> \]) - создать блок данных для анализа моделью (нормализация данных)
```
подготовить_данные_для_анализа([ ])
```
4. np.array( <Данные> ) - преобразовать информацию в вид пригодный для анализа
```
преобразовать_в_массив( )
```
5. model.predict() - выполнить предсказание по обученной модели
```
модель.предсказать( )
```

#### Программа на Псевдокоде:

1. Прочитать содержимое файла:
   file_content ← считать_файл(путь_к_файлу('AdaptiveAntivirus', 'internet_files', 'file.bak'))

2. Подготовить данные для анализа:
   file_features ← подготовить_данные_для_анализа(\[file_content\])

3. Преобразовать данные в массив:
   file_features_np ← преобразовать_в_массив(file_features)

4. Выполнить прогноз с использованием модели:
   prediction ← модель.предсказать(file_features_np)

5. Вернуть результат:
   prediction

***НЕ забывайте, что функции возвращают результат обработки информации, который необходимо поместить в переменную для последующего использования***

In [ ]:
# read_file( os.path.join( 

In [ ]:
# data_for_analysis([

In [ ]:
# np.array(

In [ ]:
# model.predict(